In [73]:
import pandas as pd

In [74]:
df=pd.read_csv('DGA-Alexa2022-label.csv')

In [75]:
df.head()

,URL,Label
0,google.com,0
1,youtube.com,0
2,baidu.com,0
3,qq.com,0
4,facebook.com,0


In [76]:
X=df.drop('Label',axis=1)

In [77]:
## Get the Dependent features
y=df['Label']

In [78]:
X.shape,y.shape

((1594795, 1), (1594795,))

In [79]:
import tensorflow as tf

In [80]:
tf.__version__

'2.8.0'

In [81]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [82]:
### Vocabulary size
voc_size=1000000

# Onehot Representation


In [83]:
urls=X.copy()

In [84]:
urls['URL'][0]

'google.com'

In [85]:
urls.reset_index(inplace=True)

In [86]:
import nltk
import re
from nltk.corpus import stopwords

In [87]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/raghavendra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [88]:
### Dataset Preprocessing
corpus = []
for i in range(0, len(urls)):
    #print(i)
    url = re.sub('[^a-zA-Z0-9]', ' ', urls['URL'][i])
    url = url.lower()
    url = url.split()
    url = ' '.join(url)
    corpus.append(url)
    #print(url)

In [89]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[341952, 363739],
 [302570, 363739],
 [509782, 363739],
 [200088, 363739],
 [497500, 363739],
 [49945, 363739],
 [636375, 363739],
 [676818, 363739],
 [98158, 363739],
 [930234, 329930],
 [355443, 850621],
 [269139, 363739],
 [503301, 363739],
 [936022, 363739, 985117],
 [847425, 363739],
 [957601, 363739],
 [318307, 363739],
 [979014, 363739],
 [324311, 363739],
 [77332, 363739],
 [298030, 363739],
 [187733, 363739],
 [634994, 363739],
 [359152, 363739],
 [890582, 363739],
 [328021, 363739],
 [356941, 363739],
 [734028, 363739],
 [513883, 363739],
 [405088, 275863],
 [264131, 363739],
 [341952, 363739, 614352],
 [242891, 363739],
 [781481, 363739],
 [935455, 363739],
 [225292, 363739],
 [98158, 889489, 622585],
 [83714, 363739],
 [558178, 363739],
 [131449, 985117],
 [261851, 333018],
 [14977, 363739],
 [2325, 363739],
 [494582, 363739],
 [999564, 363739],
 [165889, 363739],
 [840711, 363739],
 [812466, 363739],
 [636375, 413214],
 [547407, 363739],
 [636375, 889489, 622585],
 [41229

# Embedding Representation


In [90]:
sent_length=5
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[     0      0      0 341952 363739]
 [     0      0      0 302570 363739]
 [     0      0      0 509782 363739]
 ...
 [     0      0      0 362680 363739]
 [     0      0      0 775850 363739]
 [     0      0      0 614093 363739]]


In [91]:
embedded_docs[0]

array([     0,      0,      0, 341952, 363739], dtype=int32)

In [92]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 40)             40000000  
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40,056,501
Trainable params: 40,056,501
Non-trainable params: 0
_________________________________________________________________
None


In [93]:
len(embedded_docs),y.shape

(1594795, (1594795,))

In [94]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [95]:
X_final.shape,y_final.shape

((1594795, 5), (1594795,))

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=42)

In [97]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
  254/19935 [..............................] - ETA: 3:29:15 - loss: 0.5859 - accuracy: 0.6882

KeyboardInterrupt: 

# Adding Dropout


In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Performance Metrics And Accuracy


In [ ]:
from sklearn.metrics import confusion_matrix

y_pred=model.predict_classes(X_test)


In [ ]:
confusion_matrix(y_test,y_pred)


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)